#### Before You Start


cd .devcontainer </br>
run docker-compose up

In [1]:
! pip install psycopg2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-linux_x86_64.whl size=635590 sha256=c1a9ec789e8ba99161b41ee939e70779f5b40f3e60fd51cfb5ed2ebf9046aab6
  Stored in directory: /home/codespace/.cache/pip/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
import psycopg2  #import of the psycopg2 python library
import pandas as pd #import of the pandas python library
import pandas.io.sql as psql

##No transaction is started when commands are executed and no commit() or rollback() is required. 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [ ]:
try:
    # Connect to the postgreSQL server with username, and password credentials
    con = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "localhost",
                                  port = "5432")
    
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print("Connected Successfully to PostgreSQL server!!")
    
    # Obtain a DB Cursor to perform database operations
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error :
     print ("Error while connecting to PostgreSQL", error)


Connected Successfully to PostgreSQL server!!


In [ ]:
import pandas as pd
df = pd.read_csv('circuits.csv')


: 

In [16]:
print(df.head())  # Displays the first 5 rows


   circuitId   circuitRef                            name      location  \
0          1  albert_park  Albert Park Grand Prix Circuit     Melbourne   
1          2       sepang    Sepang International Circuit  Kuala Lumpur   
2          3      bahrain   Bahrain International Circuit        Sakhir   
3          4    catalunya  Circuit de Barcelona-Catalunya      Montmeló   
4          5     istanbul                   Istanbul Park      Istanbul   

     country       lat        lng  alt  \
0  Australia -37.84970  144.96800   10   
1   Malaysia   2.76083  101.73800   18   
2    Bahrain  26.03250   50.51060    7   
3      Spain  41.57000    2.26111  109   
4     Turkey  40.95170   29.40500  130   

                                                 url  
0  http://en.wikipedia.org/wiki/Melbourne_Grand_P...  
1  http://en.wikipedia.org/wiki/Sepang_Internatio...  
2  http://en.wikipedia.org/wiki/Bahrain_Internati...  
3  http://en.wikipedia.org/wiki/Circuit_de_Barcel...  
4         http://en.w

#### Create sample table

In [ ]:
#Create "test" Table

try:
    #table_name variable
    seasonTable="season"
    create_seasonTable_query = '''CREATE TABLE '''+ seasonTable+''' 
              (year INT PRIMARY KEY,
                url TEXT
               ); '''

    #Execute this command (SQL Query)
    cursor.execute(create_seasonTable_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("Table ("+ seasonTable +") created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

#### Check table creation

# [information_schema.tables] keep listing of every table being managed by Postgres for a particular database.
# specifying the tabel_schema to 'public' to only list tables that you create.
cursor.execute("""SELECT table_name 
                  FROM information_schema.tables 
                  WHERE table_schema = 'public'  
               """)

for table in cursor.fetchall():
    print(table)

#### Test Entering Values into Seasons table

In [ ]:
sql_insert_customers = "INSERT INTO season (year,url) VALUES(%s,%s)"

#list of customers (With Their attributes values)
#None here means value will be missing in the table
customer_List=[
            (1, "mohamed.ragb@ut.ee"),
            (2, "j.smith@hotmail.com"),
            (3, None),
            (4, "jeanlime@gmail.com"),
            (5,None)]

try:
    # execute the INSERT statement
    cursor.executemany(sql_insert_customers,customer_List)
    # commit the changes to the database
    con.commit()
    #the number of inserted rows/tuples
    count = cursor.rowcount
    print (count, "Record inserted successfully into customers table")

except (Exception, psycopg2.Error) as error :
    con.rollback()
    print ("Error while Inserting the data to the table, Details: ",error)

#### View outcome

In [ ]:
sql_select_query = """ SELECT * FROM season """

try:
    
    cursor.execute(sql_select_query, (1,))
    person_records = cursor.fetchall() 
    print("Print each row and it's columns values:\n")
    for row in person_records:
        print("year = ", row[0], )
        print("url = ", row[1], )
except(Exception, psycopg2.Error) as error :
    con.rollback()
    print("Error:", error)